In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json

In [2]:
cid = ''
secret = ''
sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id=cid, client_secret=secret))

In [3]:
# Example playlist: "wow im at the beach"
playlist_uri = 'spotify:playlist:5CF6KvWn85N6DoWufOjP5T'

In [4]:
offset = 0
playlist_res = []

while True:
    response = sp.playlist_items(playlist_uri,
                                 offset=offset,
                                 fields='items.track.id,items.track.artists,items.track.name,items.track.album,total',
                                 additional_types=['track'])

    if len(response['items']) == 0:
        # Combine inner lists and exit loop
        # Todo: ask how this comprehension actually works
        playlist_res = [j for i in playlist_res for j in i]
        break

    playlist_res.append(response['items'])
    offset = offset + len(response['items'])
    print(offset, "/", response['total'])

100 / 210
200 / 210
210 / 210


In [15]:
playlist_res

[{'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7hYOOcRqfNzeE2CTpajd5a'},
      'href': 'https://api.spotify.com/v1/artists/7hYOOcRqfNzeE2CTpajd5a',
      'id': '7hYOOcRqfNzeE2CTpajd5a',
      'name': 'Tabu Ley Rochereau',
      'type': 'artist',
      'uri': 'spotify:artist:7hYOOcRqfNzeE2CTpajd5a'}],
    'available_markets': ['AE',
     'AR',
     'AT',
     'AU',
     'BE',
     'BG',
     'BH',
     'CA',
     'CH',
     'CI',
     'CL',
     'CO',
     'CW',
     'CY',
     'CZ',
     'DE',
     'DK',
     'EC',
     'EE',
     'ES',
     'FI',
     'FR',
     'GB',
     'GN',
     'GR',
     'HN',
     'HU',
     'IE',
     'IL',
     'IS',
     'IT',
     'JO',
     'JP',
     'KE',
     'KR',
     'KW',
     'LB',
     'LT',
     'LU',
     'LV',
     'MA',
     'ML',
     'MT',
     'MX',
     'NE',
     'NL',
     'NO',
     'NZ',
     'OM',
     'PE',
     'PL',
     'PT',
     'QA',
     'RO',
     '

In [6]:
# Create empty df to add json response to
df = pd.DataFrame(columns=['artist', 'track_name',
                  'id', 'album'], index=range(len(playlist_res)))

In [7]:
# Add artist, album, song title, and ID to df
for i in range(len(playlist_res)):
    df['artist'].iloc[i] = playlist_res[i]['track']['artists'][0]['name']
    df['track_name'].iloc[i] = playlist_res[i]['track']['name']
    df['id'].iloc[i] = playlist_res[i]['track']['id']
    df['album'].iloc[i] = playlist_res[i]['track']['album']['name']


In [8]:
df

,artist,track_name,id,album
0,Tabu Ley Rochereau,Laiss Toi Aime',4MTiPNTHdAefzgJ7qgp3rY,Laiss Toi Aime
1,Negro Leo,Tudo Foi Feito pra Gente Lacrar,3k02yoh9Sjhjd6rKef1jyN,Desejo de Lacrar
2,Shad,Rose Garden,61kT5zIV1Z9sPbdAlKRExr,TSOL
3,Tabu Ley Rochereau,Marie Maria,2i7hxvOTfTjtsEJuWYnKg0,Laiss Toi Aime
4,Tabu Ley Rochereau,Mokrano,0gflenw36L3PVpVUqtRZJd,Laiss Toi Aime
...,...,...,...,...
205,The Cyrkle,The Visit (She Was Here),5dIcC2RTD2BOHTC1F0jrIN,Red Rubber Ball (A Collection)
206,Gilberto Gil,Era nova,6wPo638YVloLLGDPZRBE2F,Refavela
207,Paco de Lucía,Entre Dos Aguas - Instrumental,3Fgk17TfdjXWFroJy29pXH,Fuente Y Caudal
208,Inflo,No Fear,0YxMtGA6CackoDhYYmK3JT,No Fear


In [9]:
# Get audio features while avoiding rate limit

features = []
offset_min = 0
offset_max = 50

while True:
    if offset_min > len(df):
        break
    features.append(sp.audio_features(df['id'].iloc[offset_min:offset_max]))
    offset_min += 50
    offset_max += 50

In [10]:
# Combine lists within previously created list and add to new DataFrame
features = [j for i in features for j in i]
df_features = pd.DataFrame(features)

In [11]:
# Combine DataFrames
merged = df.combine_first(df_features.drop(columns=['track_href', 'analysis_url', 'uri', 'type', 'acousticness', 'danceability',
                                                    'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'valence']))

# Round tempos to nearest whole number for easier. Playlist generation works with tempo ranges, so decimal precision is unnecessary.
merged['tempo'] = round(merged['tempo']).astype(int)

In [12]:
# Convert pitch class and mode integer columns to diatonic key signature column.
def key_to_camelot(df):
    df['key'] = df['key'].astype(str).replace({'-1': 'no key detected', '0': 'C', '1': 'D-flat', '2': 'D', '3': 'E-flat', '4': 'E',
                                                   '5': 'F', '6': 'F-sharp', '7': 'G', '8': 'A-flat', '9': 'A', '10': 'B-flat', '11': 'B'})

    df['mode'] = np.where(df['mode'] == 1, 'major', 'minor')
    df['key_signature'] = df['key'] + ' ' + df['mode']
    # Dictionary for mapping key signature integer value to pitch scale. Consult https://en.wikipedia.org/wiki/Pitch_class if you would like to confirm the translation.

    key_to_wheel = {'A-flat minor': '1A', 'B major': '1B', 'E-flat minor': '2A', 'F-sharp major': '2B', 'B-flat minor': '3A', 'D-flat major': '3B',
                    'F minor': '4A', 'A-flat major': '4B', 'C minor': '5A', 'E-flat major': '5B', 'G minor': '6A', 'B-flat major': '6B',
                    'D minor': '7A', 'F major': '7B', 'A minor': '8A', 'C major': '8B', 'E minor': '9A', 'G major': '9B',
                    'B minor': '10A', 'D major': '10B', 'F-sharp minor': '11A', 'A major': '11B', 'D-flat minor': '12A', 'E major': '12B'}

    # Convert diatonic key signatures to Camelot wheel equivalents.

    df['camelot'] = df['key_signature'].map(key_to_wheel)
    df = df.drop(columns=['key', 'mode'])



In [16]:
# Load Camelot wheel, represented as json data.
with open('camelot.json') as json_file:
    camelot_json = json.load(json_file)
    wheel = camelot_json

In [17]:
key_to_camelot(merged)

In [18]:
merged

,album,artist,duration_ms,id,key,mode,tempo,time_signature,track_name,key_signature,camelot
0,Laiss Toi Aime,Tabu Ley Rochereau,254520,4MTiPNTHdAefzgJ7qgp3rY,C,major,133,4,Laiss Toi Aime',C major,8B
1,Desejo de Lacrar,Negro Leo,235882,3k02yoh9Sjhjd6rKef1jyN,A-flat,minor,124,4,Tudo Foi Feito pra Gente Lacrar,A-flat minor,1A
2,TSOL,Shad,179200,61kT5zIV1Z9sPbdAlKRExr,B,major,183,4,Rose Garden,B major,1B
3,Laiss Toi Aime,Tabu Ley Rochereau,341040,2i7hxvOTfTjtsEJuWYnKg0,B-flat,major,89,4,Marie Maria,B-flat major,6B
4,Laiss Toi Aime,Tabu Ley Rochereau,326760,0gflenw36L3PVpVUqtRZJd,A,major,130,4,Mokrano,A major,11B
...,...,...,...,...,...,...,...,...,...,...,...
205,Red Rubber Ball (A Collection),The Cyrkle,138133,5dIcC2RTD2BOHTC1F0jrIN,G,major,116,4,The Visit (She Was Here),G major,9B
206,Refavela,Gilberto Gil,238880,6wPo638YVloLLGDPZRBE2F,G,major,78,4,Era nova,G major,9B
207,Fuente Y Caudal,Paco de Lucía,361867,3Fgk17TfdjXWFroJy29pXH,E,minor,102,4,Entre Dos Aguas - Instrumental,E minor,9A
208,No Fear,Inflo,251293,0YxMtGA6CackoDhYYmK3JT,F,major,120,4,No Fear,F major,7B


In [19]:
# Replace str with track ID corresponding to the song you wish to perform queries on.

input_song_id = '6CUk0IQxiFNZRmCKP8t63o'

# Replace line above with line below for a random song.
# input_song_id = merged.iloc[randint(0, len(merged) - 1)]['id']


In [20]:

song_selected = merged.loc[merged['id'] == input_song_id]
song_selected

,album,artist,duration_ms,id,key,mode,tempo,time_signature,track_name,key_signature,camelot
28,"The Best of Franco, Vol. 2 (Dix ans après sa m...",Franco & le T.P OK Jazz,443830,6CUk0IQxiFNZRmCKP8t63o,F,minor,126,4,Kadima,F minor,4A


In [21]:
merged = merged[['track_name', 'artist', 'album', 'tempo',
                 'key_signature', 'camelot', 'time_signature', 'duration_ms', 'id']]


In [22]:
# Select harmonically compatible key signatures in camelot.json 
friendly_keys = wheel[song_selected['camelot'][song_selected.index[0]]]['all']

# Designate desired tempo range
tempo_range = 10
selected_tempo = song_selected['tempo'][song_selected.index[0]]
acceptable_tempos = list(range(selected_tempo - tempo_range, selected_tempo + tempo_range, 1))

# Show tracks with harmonically compatible key signatures within a given tempo range. Accounts for Spotify's tendency to double or halve numeric tempos.
merged.query('camelot in @friendly_keys & (tempo in @acceptable_tempos | tempo * 2 in @acceptable_tempos | tempo / 2 in @acceptable_tempos)')

,track_name,artist,album,tempo,key_signature,camelot,time_signature,duration_ms,id
10,Uptown Babies Don't Cry,Max Romeo,War Ina Babylon (Expanded Edition),129,A-flat major,4B,4,299333,1dj9IEQErJnrww3P2ceJL4
28,Kadima,Franco & le T.P OK Jazz,"The Best of Franco, Vol. 2 (Dix ans après sa m...",126,F minor,4A,4,443830,6CUk0IQxiFNZRmCKP8t63o
31,Exhibition Dechaud,Docteur Nico,Eternel Docteur Nico (Merveilles du passé 1967),123,A-flat major,4B,4,223400,4aMYJi9S00tsIdd1fij0pj
36,Venez voir,22 Band,Venez voir !!,123,C minor,5A,4,363950,5RJtuk2YXGOpgcGKhUQxuO
40,Obsessão / Não Me Diga Adeus / Pois é / A Flor...,Maria Creuza,Eu Disse Adeus,132,G minor,6A,4,278067,0Ou1IEYHdJQVPP9VI4v0Al
66,O Trem Azul,Lô Borges,Retratos,130,D-flat major,3B,4,243600,7Fd9g5y4q8ApG79VQeLfop
79,Atomic Bomb,William Onyeabor,World Psychedelic Classics 5: Who Is William O...,117,D-flat major,3B,4,479547,3YQacpdl0a7j1KbKDEUcYh
84,Lady of Leisure,Gifted Gab,Gab the Most High,123,A-flat major,4B,4,196235,2paj5c87oBVW909huOF92v
95,LITE SPOTS,KAYTRANADA,99.9%,120,D-flat major,3B,4,230920,3Fc7k96EGOGiJBMZUxbpq7
134,The Answer,Marcos Valle,Samba '68,120,E minor,9A,4,165267,6lF9V3yNpDyyDmYJx1gzE3
